In [10]:
pip --version

pip 20.2 from c:\users\gaukharjavarova\appdata\local\programs\python\python38-32\lib\site-packages\pip (python 3.8)

Note: you may need to restart the kernel to use updated packages.


In [9]:
!pip install matplotlib

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\users\\gaukharjavarova\\appdata\\local\\programs\\python\\python38-32\\Lib\\site-packages\\matplotlib\\_image.cp38-win32.pyd'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\users\gaukharjavarova\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.



  Using cached matplotlib-3.3.1-1-cp38-cp38-win32.whl (8.7 MB)


In [7]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from scipy.stats.stats import ttest_ind
import warnings

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

%matplotlib inline

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'lifeexpectancy'
table_name = 'lifeexpectancy'
conn_str = f"postgresql://{postgres_user}:{postgres_pw}@{postgres_host}:{postgres_port}/{postgres_db}"
        
query = """
SELECT *
FROM lifeexpectancy
"""

ImportError: cannot import name 'animation' from partially initialized module 'matplotlib' (most likely due to a circular import) (c:\users\gaukharjavarova\appdata\local\programs\python\python38-32\lib\site-packages\matplotlib\__init__.py)

In [8]:
life_expectancy = pd.read_sql_query(query, con=conn_str)
life_expectancy

NameError: name 'query' is not defined

In [ ]:
life_expectancy.info()

In [ ]:
life_expectancy.isna().mean()

#can fill in the missing values with mean() or mode() for columns where NAs are <5 and others with interpolate
#after eliminating the outliers
#Missing values in Hepatitis B = 19%, big missing values in Population 22%, GDP 15%, BMI 11%
#- it's all understandable, since we have 83% of developing countries compared to 17% of developed.  

In [ ]:
sns.boxplot(life_expectancy["Life expectancy"])
plt.show()

In [ ]:
sns.distplot(life_expectancy["Population"])
plt.show()

In [ ]:
sns.violinplot(life_expectancy["Population"])
plt.show()

obviously see some outliers from plots presented above on Population column, let's confirm it through the Zscore below:

In [ ]:
#scale the var so that its mean is 0 and std is 1. then measure how far a value is far away from the mean(0) in terms of std(1)
#the bigger the Zscore the father is from the mean = outlier. 
#here i see 3, 4 are outliers probably, so i can drop them
#Formula for Zscore = (Observation-Mean)/STD
#z = (X - u) / o

from scipy.stats import zscore

z_scores = zscore(life_expectancy["Population"])
for threshold in range(1,5):
    print("The score threshold is: {}".format(threshold))
    print("The indices of the outliers:")
    print(np.where(z_scores > threshold))
    print("Number of outliers is: {}".format(len((np.where(z_scores > threshold)[0]))))

In [ ]:
life_expectancy["Population"] = life_expectancy["Population"].interpolate()


In [ ]:
life_expectancy["Schooling"] = life_expectancy["Schooling"].interpolate()
life_expectancy["Total expenditure"] = life_expectancy["Total expenditure"].interpolate()
life_expectancy["GDP"] = life_expectancy["GDP"].interpolate()


In [ ]:
life_expectancy["Hepatitis B"] = life_expectancy["Hepatitis B"].mean()
life_expectancy["Alcohol"] = life_expectancy["Alcohol"].mean()
life_expectancy["Income composition of resources"] = life_expectancy["Income composition of resources"].interpolate()


In [ ]:
life_expectancy.describe()

In [ ]:
life_expectancy.info()

In [ ]:
life_expectancy.isnull().sum()*100/life_expectancy.count()

I'm satisfied with filled missing values now. Used interpolate() approach with time series, expecting continuity in variables that evolves over time. 

In [ ]:
sns.scatterplot("Population", "Hepatitis B", 'Income composition of resources', data=life_expectancy)
plt.show()

OK, we can see from here, that income composition growth plays a big role in a population growth

In [ ]:
plt.figure(figsize=(13, 9))
sns.heatmap(life_expectancy.corr(), vmin=-1, vmax=1, annot=False)
plt.show()

Lighter colors indicate a higher correlation between variables. Wow, these stats show the strong correlations between schooling and life expectancy! I actually didn't expect it, taking into consideration that most countries in our dataset are in 'developing' category. Not many people can afford going to school there, here we see that people who get to go to school live longer! With schooling the income composition of resources is higher too. 

In [ ]:
import scipy.stats as stats
from scipy.stats.mstats import winsorize

In [ ]:
col_to_winsorize = life_expectancy[
    [
        "Life expectancy ",
        "Adult Mortality",
        "Alcohol",
        " BMI ",
        "infant deaths",
        "percentage expenditure",
        "Measles ",
        "under-five deaths ",
        "Income composition of resources",
        "Schooling",
    ]
]
for col in col_to_winsorize:
    life_expectancy[col] = winsorize(life_expectancy[col], (0.1, 0.1))

In [ ]:
life_expect_nan = life_expectancy.drop(['Country', 'Year', 'Status'], axis=1)


In [ ]:
col_to_transform = life_expect_nan.drop(
    [
        "Life expectancy ",
        "Adult Mortality",
        "Alcohol",
        " BMI ",
        "infant deaths",
        "percentage expenditure",
        "Measles ",
        "under-five deaths ",
        "Income composition of resources",
        "Schooling",
    ],
    axis=1,
)
for col in col_to_transform:
    life_expectancy[col] = np.log(life_expectancy[col])

In [ ]:
for col in life_expect_nan:
    sns.boxplot((life_expectancy[col]))
    plt.show()

multivariate exploration techniques

In [ ]:
life_expectancy.describe(include=["O"])


In [ ]:
life_expectancy.describe(include= 'all')

In [ ]:
plt.figure(figsize=(18,15))


    histograms of the original data
plt.subplot(3, 3, 1)
plt.hist(life_expectancy["Schooling"])
plt.title("histogram of schooling")

plt.subplot(3, 3, 2)
plt.hist(life_expectancy["Income composition of resources"])
plt.title("histogram of Income composition of resources")

plt.subplot(3, 3, 3)
plt.hist(life_expectancy["percentage expenditure"])
plt.title("histogram of percentage expenditure")

     histograms of the winsorized data
plt.subplot(3, 3, 4)
plt.hist(life_expectancy["winsorized_schooling"])
plt.title("histogram of Schooling (winsorized)")

plt.subplot(3, 3, 5)
plt.hist(life_expectancy["winsorized_Income composition of resources"])
plt.title("histogram of Income composition of resources (winsorized)")

plt.subplot(3, 3, 6)
plt.hist(life_expectancy["winsorized_percentage expenditure"])
plt.title("histogram of percentage expenditure (winsorized)")

     histograms of the log transformed data
plt.subplot(3, 3, 7)
plt.hist(np.log(life_expectancy["Schooling"]))
plt.title("histogram of Schooling(log transformed)")

plt.subplot(3, 3, 8)
plt.hist(np.log(life_expectancy["Income composition of resources"]))
plt.title("histogram of Income composition of resources (log transformed)")

plt.subplot(3, 3, 9)
plt.hist(np.log(life_expectancy["percentage expenditure"]))
plt.title("histogram of percentage expenditure (log transformed)")

plt.show()

In [ ]:

life_expectancy.corr()

Feature Engineering

In [ ]:
import numpy as np
import pandas as pd

from scipy import stats
import statsmodels.api as sm
from sqlalchemy import create_engine

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'lifeexpectancy'
table_name = 'lifeexpectancy'
conn_str = f"postgresql://{postgres_user}:{postgres_pw}@{postgres_host}:{postgres_port}/{postgres_db}"
        
query = """
SELECT *
FROM lifeexpectancy
"""

#did this to start with the original data = NAs

In [ ]:
def show_top_missing(df, n=5, rate=True):
    life_expectancy = df.isna()
    
    # Aggregate the df:
    # * if rate is true, then calculate what percent of values are nan
    # * if rate is false, then calculate the count of nans
    if rate:
        missing_stats = life_expectancy.mean()
    else:
        missing_stats = life_expectancy.sum()

    top_missing = missing_stats.sort_values(ascending=False)
    print(top_missing.head(n))


In [ ]:
show_top_missing(life_expectancy)
life_expectancy.head()

In [ ]:
life_expectancy.corr().tail(1)

In [ ]:
sns.countplot(life_expectancy["Total expenditure"])
plt.show()

In [ ]:
sns.boxplot("Income composition of resources", "Schooling", data=life_expectancy)
plt.show()

clearly see that schooling affects the income growth

!pip3 install -U scikit-learn scipy matplotlib


In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
X = life_expectancy.drop('Country', axis=1)

# Store what you want to predict () in y
y = life_expectancy["Life expectancy "]

In [ ]:

print("Feature variances before scaling:")
print(X.var().sort_values(ascending=False).head())

scaler = StandardScaler()

    'learn' from the data
scaler.fit(X)

    apply what the scaler learned
scaled_X = scaler.transform(X)

scaled_df = pd.DataFrame(scaled_X, columns=X.columns)

print("\n\nFeature variances before scaling:")
print(scaled_df.var().sort_values(ascending=False).head())

In [ ]:
pca = PCA()

    'learn' from the data
pca.fit(scaled_X)

    apply what the pca object learned
pca_X = pca.transform(scaled_X)

     What attribute holds the variance explained?
plt.plot(pca.explained_variance_ratio_, marker="o")
plt.title("Variance explained by each component")
plt.show()

sns.heatmap(pd.DataFrame(pca_X).corr(), vmin=-1, vmax=1)
plt.title("Correlation between components")
plt.show()

I don't know what's wrong with it, but will work on it more. This is crazy, i'm sitting on this challenge for more than 20 hours, hopefully it will get better and quicker over time. 